<a href="https://colab.research.google.com/github/agarwal-peeush/Learning/blob/PadhAI_learning/Learning-Python/padhAI/MPNeuron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sklearn.datasets
import numpy as np

In [0]:
breast_cancer = sklearn.datasets.load_breast_cancer()

In [0]:
X = breast_cancer.data
Y = breast_cancer.target

In [0]:
print(X)
print(Y)

In [0]:
print(X.shape, Y.shape)

In [0]:
import pandas as pd

In [0]:
data = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)

In [0]:
data['class'] = breast_cancer.target

In [0]:
data.head()

In [0]:
data.describe()

In [0]:
data['class'].value_counts()

In [0]:
print(breast_cancer.target_names)

In [0]:
data.groupby('class').mean()